In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
device="cuda" if torch.cuda.is_available() else "cpu"

In [4]:
model_name= "gpt2-xl"

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
tokenizer=AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

 **At each timestep, we pick out the
model’s logits for the last token in the prompt and wrap them with a softmax to get a
probability distribution. We then pick the next token with the highest probability, add
it to the input sequence, and run the process again.**

In [6]:
import pandas as pd

In [7]:
input_text="Unicorns are "
input_ids=tokenizer(input_text,return_tensors="pt")["input_ids"].to(device)


iterations = []
n_steps = 14
choices_per_step = 5

with torch.no_grad():
    for _ in range(n_steps):
        iteration = dict()
        iteration["Input"] = tokenizer.decode(input_ids[0])

        output = model(input_ids=input_ids)

        # Select logits of the first batch and the last token and apply softmax
        next_token_logits = output.logits[0, -1, :]
        next_token_probs = torch.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)

        # Store tokens with highest probabilities
        for choice_idx in range(choices_per_step):
            token_id = sorted_ids[choice_idx]
            token_prob = next_token_probs[token_id].cpu().numpy()
            token_choice = (
                f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%)"
            )
            iteration[f"Choice {choice_idx+1}"] = token_choice

        # Append predicted next token to input for the next loop iteration
        input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)

        iterations.append(iteration)

# Display the final results in a DataFrame
pd.DataFrame(iterations)


,Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,Unicorns are,(31.81%),~~ (7.39%),urs (6.51%),icky (6.12%),________ (4.08%)
1,Unicorns are,a (4.75%),the (3.95%),not (3.23%),f (1.24%),very (1.16%)
2,Unicorns are a,type (3.82%),(3.54%),group (3.26%),very (3.13%),popular (1.58%)
3,Unicorns are a type,of (99.18%),(0.16%),(0.13%),or (0.05%),in (0.05%)
4,Unicorns are a type of,mythical (9.65%),unicorn (7.43%),magical (6.51%),dragon (3.09%),(2.71%)
5,Unicorns are a type of mythical,creature (51.26%),beast (6.63%),animal (5.69%),", (3.05%)",horse (3.01%)
6,Unicorns are a type of mythical creature,in (23.44%),that (15.12%),. (9.59%),", (9.47%)",from (8.57%)
7,Unicorns are a type of mythical creature in,the (20.55%),Greek (5.80%),mythology (5.79%),Chinese (4.96%),ancient (3.64%)
8,Unicorns are a type of mythical creature in the,world (6.75%),(6.69%),mythology (4.46%),fantasy (2.80%),Dungeons (2.55%)
9,Unicorns are a type of mythical creature in t...,of (97.21%),. (0.48%),(0.28%),", (0.28%)",'s (0.19%)


In [8]:
input_ids = tokenizer(input_text, return_tensors="pt")["input_ids"].to(device)
output = model.generate(input_ids, max_new_tokens=n_steps, do_sample=False)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [9]:
print(tokenizer.decode(output[0]))

Unicorns are  a type of mythical creature in the world of  Dungeons and Dragons


In [10]:
max_length = 128
input_txt = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n
"""
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length,
 do_sample=False)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [11]:
print(tokenizer.decode(output_greedy[0]))

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of California, Davis, and the University of Colorado, Boulder, were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest trees.


The researchers were surprised to find that the unicorns were able to communicate with each other, and even with humans.


The researchers were surprised to find that the unicorns were able


**Beam search
decoding.**

In [12]:
import torch.nn.functional as F
def log_probs_from_logits(logits, labels):
 logp = F.log_softmax(logits, dim=-1)
 logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
 return logp_label

**This gives us the log probability for a single token, so to get the total log probability
of a sequence we just need to sum the log probabilities for each token:**

In [13]:
def sequence_logprob(model, labels, input_len=0):
  with torch.no_grad():
    output = model(labels)
    log_probs = log_probs_from_logits(
    output.logits[:, :-1, :], labels[:, 1:])
    seq_log_prob = torch.sum(log_probs[:, input_len:])
  return seq_log_prob.cpu().numpy()

In [16]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5,
 do_sample=False)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The discovery of the unicorns was made by a team of scientists from the University of California, Santa Cruz, and the National Geographic Society.


The scientists were conducting a study of the Andes Mountains when they discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English

log-prob: -55.23


stopping repetitions

In [19]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5,
 do_sample=False, no_repeat_ngram_size=2)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [20]:
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The discovery was made by a team of scientists from the University of California, Santa Cruz, and the National Geographic Society.

According to a press release, the scientists were conducting a survey of the area when they came across the herd. They were surprised to find that they were able to converse with the animals in English, even though they had never seen a unicorn in person before. The researchers were

log-prob: -93.12


**Sampling using Temperature control**

In [22]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True,
 temperature=0.5, top_k=0)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [23]:
print(tokenizer.decode(output_temp[0]))

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The valley is known as the "Valley of the Unicorns" due to the presence of a herd of the mythical creatures.

The researchers believe that the unicorns are the descendants of a small group of people who left the Andes Mountains and crossed the Andes into Argentina. The valley is located in the mountain range known as the Cordillera Blanca.

The valley is named


**Lets to top K sampling**

In [24]:
output_topk = model.generate(input_ids, max_length=max_length, do_sample=True,
 top_k=50)
print(tokenizer.decode(output_topk[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The valley was an "almost completely empty, steep rock quarry between 2000 and 3000 meters". This makes it the first place where unicorns' descendants could live in the wild in this part of the world, according to ScienceDaily.

Advertisement

The land which the unicorns' breed was originally from, had been destroyed by glaciers, but thanks to the glacier melting, a new valley was created


In [26]:
logp = sequence_logprob(model, output_topk, input_len=len(input_ids[0]))
print(f"\nlog-prob: {logp:.2f}")


log-prob: -200.84
